In [1]:
import os
from bs4 import BeautifulSoup
import requests
import time
from datetime import datetime
import json
from zipfile import ZipFile
import pandas as pd
import shutil
import zipfile
import random
# from kafka import KafkaProducer

# Producer

In [2]:
def serializer(message):
    return json.dumps(message).encode('utf-8')
# Thêm địa chỉ của tất cả các broker vào danh sách dưới đây
bootstrap_servers = ['localhost:9092', 'localhost:9093', 'localhost:9094']

# Key serializer
def key_serializer(key):
    return str(key).encode('utf-8')



# # Kafka Producer with custom partitioner
# producer = KafkaProducer(
#     bootstrap_servers=bootstrap_servers,
#     value_serializer=serializer,
#     key_serializer=key_serializer,
# )


# producer.send('user', value=dummy_message, key = random.choice(range(0,5)))


### Define PATH constant

In [3]:
BASE_PATH = "craw"
HTML_PATH = BASE_PATH + "/html"
USER_PATH = BASE_PATH + "/users"

# 1. Get Clubs IDs

In [4]:
def get_number(string):
    return int(string.strip().replace(",", ""))


clubs_id = set()
possibles_users = 0
page = 1

while True:
    print(f"\r{page}", end="")

    time.sleep(3)  # Wait 3 seconds per page
    data = requests.get(f"https://myanimelist.net/clubs.php?p={page}")
    soup = BeautifulSoup(data.text, "html.parser")
    rows = soup.find_all("tr", {"class": "table-data"})
    for row in rows:
        members = get_number(row.find("td", {"class": "ac"}).text)
        club_id = get_number(
            row.find("a", {"class": "fw-b"}).get("href").split("=")[-1]
        )
        if (
            club_id not in clubs_id and members > 30
        ):  # Only save groups with more than 30 members
            possibles_users += members
            clubs_id.add(club_id)

    page += 1
    # if possibles_users > 1000000:  # Threshold to stop
    if possibles_users > 10:  # Threshold to stop
        break

# Create the directory if it doesn't exist
os.makedirs(BASE_PATH, exist_ok=True)

# Save the club IDs to the file
with open(os.path.join(BASE_PATH, "clubs.txt"), "w") as file:
    for club in clubs_id:
        file.write(f"{club}\n")

1

# 2. Get usernames in every clubs

In [5]:
if not os.path.exists(f"{BASE_PATH}/users_list.txt"):
    with open(f"{BASE_PATH}/users_list.txt", "w", encoding="UTF-8") as file:
        pass
    
if not os.path.exists(f"{BASE_PATH}/_revised_clubs.txt"):
    with open(f"{BASE_PATH}/_revised_clubs.txt", "w", encoding="UTF-8") as file:
        pass

In [6]:
with open(f"{BASE_PATH}/clubs.txt") as file:
    clubs_id = [x.strip() for x in file.readlines()]

with open(f"{BASE_PATH}/users_list.txt", encoding="UTF-8") as file:
    users = set([x.strip() for x in file.readlines()])

with open(f"{BASE_PATH}/_revised_clubs.txt", encoding="UTF-8") as file:
    revised_clubs = set([int(x.strip()) for x in file.readlines()])

len(users), len(revised_clubs), len(clubs_id)

(0, 0, 42)

In [7]:
#%%notify -m "get username finish"

for i, club_id in enumerate(clubs_id):
    if club_id in revised_clubs:
        continue

    # page = 1
    # while True:
    print(f"\r{i+1}/{len(clubs_id)} --> {str(page).zfill(2)}", end="")
    # link = f"https://api.jikan.moe/v3/club/{club_id}/members/{page}"
    link = f"https://api.jikan.moe/v4/clubs/{club_id}/members"

    try:
        time.sleep(4.2)
        data = requests.get(link)
    except KeyboardInterrupt:
        raise KeyboardInterrupt()
    except:  # Other exception wait 2 min and try again
        time.sleep(120)
        continue

    if data.status_code != 200:
        break

    with open(f"{BASE_PATH}/users_list.txt", "a", encoding="UTF-8") as file:
        for user in map(lambda x: x["username"], json.loads(data.text)["data"]):
            if user not in users and user != "":
                file.write(f"{user}\n")
                users.add(user)
        # page += 1

    revised_clubs.add(club_id)
    with open(f"{BASE_PATH}/_revised_clubs.txt", "a", encoding="UTF-8") as file:
        file.write(f"{club_id}\n")

42/42 --> 02

In [8]:
with open(f"{BASE_PATH}/users_list.txt", encoding="UTF-8") as file:
    users = list(set([x.strip() for x in file.readlines()]))[1:]
    random.shuffle(users)

with open(f"{BASE_PATH}/users.csv", "w", encoding="UTF-8") as file:
    file.write("user_id,username\n")
    for i, user in enumerate(users):
        file.write(f"{i},{user}\n")

# 3. Get animelist per user

In [9]:
with open(f"{BASE_PATH}/users.csv", "r", encoding="UTF-8") as file:
    file.readline()
    users = [x.strip().split(",") for x in file.readlines()]
    users = [(int(x[0]), x[1]) for x in users]

last_revised_users = -1
if os.path.exists(f"{BASE_PATH}/_last_revised_users.txt"):
    with open(f"{BASE_PATH}/_last_revised_users.txt", "r", encoding="UTF-8") as file:
        last_revised_users = int(file.readline())

len(users), last_revised_users

(1270, -1)

In [10]:
#%%notify -m "animelist finish"

USER_PATH = os.path.join(BASE_PATH, "users")  # Specify the user directory

# Create the directory if it doesn't exist
os.makedirs(USER_PATH, exist_ok=True)

for i, (user_id, username) in enumerate(users):
    if user_id <= last_revised_users:
        continue

    now = datetime.now()
    print(f'\r{str(now).split(".")[0]} --> {i+1}/{len(users)}', end="")
    page = 1
    all_animes = []
    offset = 0
    while True:
    # link = f"https://api.jikan.moe/v4/user/{username}/animelist/all?page={page}"
        link = f"https://myanimelist.net/animelist/{username}/load.json?offset={offset}"
        try:
            time.sleep(4.2)
            data = requests.get(link, timeout=15)
        except KeyboardInterrupt:
            raise KeyboardInterrupt()
        except:  # Other exception wait 2 min and try again
            time.sleep(120)
            continue


        # if data.status_code != 200:
        #     break

        data = json.loads(data.text)

        for anime in data:
            try:
                all_animes.append((anime["anime_id"], anime["score"], anime["status"], anime["num_watched_episodes"]))
            except:
                print(username)

        offset += 300
        if len(data) < 300:
            break

    if len(all_animes) != 0:
        
        user_file_path = os.path.join(USER_PATH, f"{user_id}.csv")
        with open(user_file_path, "w") as f1:
            f1.write(f"anime_id,score,watching_status,watched_episodes\n")
            for anime_id, anime_score, watching_status, watched_episodes in all_animes:
                f1.write(
                    f"{anime_id},{anime_score},{watching_status},{watched_episodes}\n"
                )

    revised_users = user_id
    with open(f"{BASE_PATH}/_last_revised_users.txt", "w", encoding="UTF-8") as file:
        file.write(f"{user_id}\n")

2023-12-17 08:04:37 --> 8/1270Acosmist
2023-12-17 08:07:58 --> 29/1270Amelix
2023-12-17 08:11:21 --> 46/1270-Auria-
2023-12-17 08:14:42 --> 63/1270Infamous_Empire
2023-12-17 08:21:00 --> 97/1270-cutthroat954
2023-12-17 08:24:45 --> 118/1270Beatrize
2023-12-17 08:31:19 --> 147/1270AdmiralScarlet-
2023-12-17 08:41:01 --> 194/1270123114334d
2023-12-17 08:43:58 --> 210/1270Glazuelo
2023-12-17 08:44:54 --> 214/1270-Roxana
2023-12-17 08:45:21 --> 218/1270-Azazel-san
2023-12-17 08:54:56 --> 273/1270Bloodylily703
2023-12-17 08:56:19 --> 283/1270-Bass
2023-12-17 09:01:10 --> 306/1270Chellle
2023-12-17 09:09:01 --> 338/1270Anhicek
2023-12-17 09:12:40 --> 356/1270-aka-
2023-12-17 09:13:11 --> 361/1270000mohit
2023-12-17 09:13:27 --> 363/1270-lolitsun
2023-12-17 09:15:10 --> 376/1270-Luk134-
2023-12-17 09:19:00 --> 390/1270Aqsul
2023-12-17 09:20:50 --> 398/1270afganiasakrisna
2023-12-17 09:21:34 --> 402/1270AfterHours
2023-12-17 09:23:19 --> 409/1270Sereshay
2023-12-17 09:24:57 --> 416/1270AnimeFa

# 4. Download Anime HTML

In [11]:
unique_anime = set()
folder = os.listdir(USER_PATH)
for i, user_file in enumerate(folder):
    if ".csv" not in user_file:
        continue

    print(f"\r{i + 1}/{len(folder)}", end="")
    with open(f"{USER_PATH}/{user_file}", "r") as file:
        file.readline()
        for line in file:
            anime = line.strip().split(",")[0]
            unique_anime.add(anime)

print("         ")
print(len(unique_anime))

1144/1144         
18217


In [12]:
MAX = 7  # MAX SECOND TO WAIT PER REQUEST
MIN = 4  # MIN SECONDS TO WAIT PER REQUEST


def sleep():
    time_to_sleep = random.random() * (MAX - MIN) + MIN
    time.sleep(time_to_sleep)


def get_link_by_text(soup, anime_id, text):
    links = list(filter(lambda x: anime_id in x["href"], soup.find_all("a", text=text)))
    return links[0]["href"]


def save(path, data):
    with open(path, "w", encoding="UTF-8") as file:
        file.write(data)


def save_link(link, anime_id, name):
    sleep()
    path = f"{HTML_PATH}/{anime_id}/{name}.html"
    data = requests.get(link)
    soup = BeautifulSoup(data.text, "html.parser")
    soup.script.decompose()
    save(path, soup.prettify())
    return soup


def save_reviews(link, anime_id):
    page = 1
    while True:
        sleep()
        actual_link = f"{link}?p={page}"
        data = requests.get(actual_link)
        soup = BeautifulSoup(data.text, "html.parser")
        reviews = soup.find_all("a", text="Overall Rating")
        if len(reviews) == 0:
            break

        path = f"{HTML_PATH}/{anime_id}/reviews_{page}.html"
        soup.script.decompose()
        save(path, soup.prettify())
        page += 1


def scrap_anime(anime_id):
    path = f"{HTML_PATH}/{anime_id}"
    os.makedirs(path, exist_ok=True)
    sleep()
    data = requests.get(f"https://myanimelist.net/anime/{anime_id}")

    anime_info = data.text
    soup = BeautifulSoup(anime_info, "html.parser")
    soup.script.decompose()
    save(f"{HTML_PATH}/{anime_id}/details.html", soup.prettify())

    link_review = get_link_by_text(soup, anime_id, "Reviews")
    link_recomendations = get_link_by_text(soup, anime_id, "Recommendations")
    link_stats = get_link_by_text(soup, anime_id, "Stats")
    link_staff = get_link_by_text(soup, anime_id, "Characters & Staff")
    link_pictures = get_link_by_text(soup, anime_id, "Pictures")

    save_link(link_pictures, anime_id, "pictures")
    save_link(link_staff, anime_id, "staff")
    save_link(link_stats, anime_id, "stats")
    save_link(link_recomendations, anime_id, "recomendations")
    save_reviews(link_review, anime_id)


def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(
                os.path.join(root, file),
                os.path.relpath(os.path.join(root, file), path),
            )


In [13]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [14]:
#%%notify -m "Anime scrapping finish"

for i, anime_id in enumerate(unique_anime):
    if os.path.isfile(f"{HTML_PATH}/{anime_id}.zip"):
        continue

    print(f"\r{i+1}/{len(unique_anime)}", end="")

    try:
        scrap_anime(anime_id)
    except KeyboardInterrupt:
        break
    except:  # Other exception wait 2 min and try again
        time.sleep(120)
        continue

    path = f"{HTML_PATH}/{anime_id}"
    zipf = zipfile.ZipFile(f"{path}.zip", "w", zipfile.ZIP_DEFLATED)
    zipdir(path, zipf)
    zipf.close()

    shutil.rmtree(path)

1/18217

C:\Users\ADMIN\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\ADMIN\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.


292/18217